<a href="https://colab.research.google.com/github/pipeberrio/Analitica/blob/main/Tarea4_Analitica_Felipe_Berrio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="banner" height="252px" width="1080px" src="https://docs.google.com/uc?export=download&id=1SqbMIjMfsMlSukiTyfMJ0VMuDlP2hGhx"  align="center" hspace="10px" vspace="0px" ></p>

# <font color='056938'> **Descripción del problema** </font>

La Autoridad Nacional de Licencias Ambientales ha consolidado y preprocesado la siguiente base de datos respecto a los trabsformadores de material reciclado en Colombia.










In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


!gdown 1oP0ygviQAsthXGlzTJnknd72F6mHPhUV
df = pd.read_csv('datos_transformadores.csv')
df.head()


La siguiente función permite visulizar para los distintos transformadores clasificados por material y según la capacidad de transformación `[alta, media o baja]`





In [ ]:

# Calculate quartiles (Q1, Q2, and Q3)
q1 = np.round(df['capacidad'].quantile(0.25), 1)  # First quartile (25th percentile)
q2 = np.round(df['capacidad'].quantile(0.50), 1)  # Median (50th percentile)
q3 = np.round(df['capacidad'].quantile(0.75), 1)  # Third quartile (75th percentile)



import plotly.graph_objs as go

def create_map(capacity):
    """
    Creates a map visualization based on the specified capacity filter.

    Parameters:
    -----------
    capacity : str
        The filter for capacity levels to include in the map:
        - 'todos' : Include all data points.
        - 'alta' : Include data points with capacity greater than or equal to the third quartile (q3).
        - 'media' : Include data points with capacity between the first (q1) and third quartiles (q3).
        - 'baja' : Include data points with capacity less than the first quartile (q1).

    Returns:
    --------
    map : plotly.graph_objs._figure.Figure
        A Plotly Scattermapbox figure displaying the selected data points with markers.
    """

    # Filter data based on capacity level
    if capacity == 'todos':
        data = df
    elif capacity == "alta":
        data = df[df['capacidad'] >= q3]
    elif capacity == "media":
        data = df[(df['capacidad'] < q3) & (df['capacidad'] >= q1)]
    elif capacity == "baja":
        data = df[df['capacidad'] < q1]

    data.reset_index(inplace=True)

    # Define list of materials
    materials = ['cartón', 'metales', 'multimaterial', 'papel', 'plástico', 'vidrio', 'varios']

    # Initialize the map
    map = go.Figure()

    # Add map traces for each material type
    for material in materials:
        map.add_trace(go.Scattermapbox(
            lat=data['latitud'],
            lon=data['longitud'],
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=12,
                opacity=0.5
            ),
            text=data['departamento'],
            hoverinfo='text',
            name=material
        ))

    # Update layout with map style and layout settings
    map.update_layout(
        mapbox_style="open-street-map",
        autosize=True,
        hovermode='closest',
        showlegend=True,
        height=600
    )

    # Set map center and zoom level
    map.update_mapboxes(
        center=go.layout.mapbox.Center(
            lat=df.loc[0, 'latitud'],
            lon=df.loc[0, 'longitud']
        ),
        zoom=4
    )

    return map


In [ ]:
create_map("baja")

## <font color='8EC044'> **Ejercicio 1** </font>

Diseñe una tablero que pemrita al usuario visualizar la información de la capacidad de los transformadores de los distintos materiales, según se muestra en el siguiente video


[![](https://docs.google.com/uc?export=download&id=1eTeCdx6gvhtn3a9kWQUURiopcoQNPkjg)](https://drive.google.com/file/d/13RdaJiVH1LcGbbF303D3HYmrr709qG24/view?usp=sharing "Introducción")




In [ ]:
# Instalar bibliotecas necesarias
!pip install dash plotly pandas flask-ngrok

import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from flask_ngrok import run_with_ngrok

# Cargar datos
!gdown 1oP0ygviQAsthXGlzTJnknd72F6mHPhUV
df = pd.read_csv('datos_transformadores.csv')

# Calcular cuartiles de capacidad
q1 = np.round(df['capacidad'].quantile(0.25), 1)
q2 = np.round(df['capacidad'].quantile(0.50), 1)
q3 = np.round(df['capacidad'].quantile(0.75), 1)

# Inicializar la aplicación Dash
app = Dash(__name__)
run_with_ngrok(app)  # Habilitar ngrok para el servidor de Dash en Colab

# Función para crear el mapa
def create_map(df, capacity):
    if capacity == 'todos':
        data = df
    elif capacity == "alta":
        data = df[df['capacidad'] >= q3]
    elif capacity == "media":
        data = df[(df['capacidad'] < q3) & (df['capacidad'] >= q1)]
    elif capacity == "baja":
        data = df[df['capacidad'] < q1]

    # Crear el mapa
    map_fig = go.Figure()
    for material in df['material'].unique():
        df_material = data[data['material'] == material]
        map_fig.add_trace(go.Scattermapbox(
            lat=df_material['latitud'],
            lon=df_material['longitud'],
            mode='markers',
            marker=go.scattermapbox.Marker(size=12, opacity=0.5),
            text=df_material['departamento'],
            hoverinfo='text',
            name=material
        ))

    # Configuración del mapa centrado en Colombia
    map_fig.update_layout(
        mapbox_style="open-street-map",
        autosize=True,
        hovermode='closest',
        showlegend=True,
        height=600,
        mapbox=dict(
            center=go.layout.mapbox.Center(
                lat=4.5709,  # Latitud aproximada del centro de Colombia
                lon=-74.2973  # Longitud aproximada del centro de Colombia
            ),
            zoom=4  # Ajusta el zoom para ver todo el país
        )
    )

    return map_fig

# Layout de la aplicación
app.layout = html.Div([
    html.H1("Transformadores de Material Reciclado en Colombia"),
    html.Label("Selecciona el nivel de capacidad:"),
    dcc.Dropdown(
        id='capacity-dropdown',
        options=[
            {'label': 'Todos', 'value': 'todos'},
            {'label': 'Alta capacidad', 'value': 'alta'},
            {'label': 'Media capacidad', 'value': 'media'},
            {'label': 'Baja capacidad', 'value': 'baja'}
        ],
        value='todos'
    ),
    dcc.Graph(id='map-graph')
])

# Callback para actualizar el mapa en función del filtro de capacidad
@app.callback(
    Output('map-graph', 'figure'),
    [Input('capacity-dropdown', 'value')]
)
def update_map(selected_capacity):
    return create_map(df, selected_capacity)

# Ejecutar la aplicación
app.run_server()
